In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [112]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [113]:
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true


In [114]:
cv_only = True
save_cv = True
full_train = False

In [115]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True


In [116]:
path = '../input/'

In [228]:
NROWS = 50000

In [229]:
train = pd.read_csv(path + 'train.csv', nrows=NROWS)

In [230]:
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path + 'test.csv', nrows=NROWS)
test_id = test['id']

In [231]:
NFOLDS = 5
kfold = StratifiedKFold(n_splits = NFOLDS, shuffle=True, random_state=21)

In [232]:
y = train['target'].values
drop_feature = [
    'id',
    'target'
]

In [233]:
X= train.drop(drop_feature, axis=1)

In [234]:
feature_names = X.columns.tolist()
cat_features = [c for c in feature_names if ('cat' in c and 'cound' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' in c)]

calc는 도움이 안되기 때문에 다 빼버린다

## Feature Engineering

In [235]:
(train == -1).sum(axis=1)

0        1
1        2
2        3
3        0
4        2
5        1
6        1
7        0
8        1
9        0
10       2
11       2
12       2
13       0
14       2
15       2
16       2
17       0
18       3
19       2
20       2
21       2
22       0
23       2
24       1
25       2
26       1
27       1
28       2
29       1
        ..
49970    2
49971    0
49972    2
49973    2
49974    3
49975    0
49976    1
49977    2
49978    2
49979    1
49980    0
49981    0
49982    2
49983    2
49984    2
49985    2
49986    0
49987    0
49988    0
49989    2
49990    3
49991    1
49992    0
49993    2
49994    2
49995    1
49996    1
49997    3
49998    2
49999    3
Length: 50000, dtype: int64

In [236]:
pd.concat([(train == -1).sum(axis=1),train['target']], axis=1).groupby(0).mean()

,target
0,
0,0.046912
1,0.037062
2,0.033452
3,0.026276
4,0.039301
6,0.666667
7,0.333333


In [237]:
train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)

In [238]:
num_features.append('missing')

In [239]:
for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] =le.transform(test[c])

In [240]:
enc = OneHotEncoder()
enc.fit(train[cat_features])

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [241]:
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

In [242]:
ind_features = [c for c in feature_names if 'ind' in c]
count = 0
for c in ind_features:
    if count==0:
        train['new_ind'] = train[c].astype(str)+'_'
        test['new_ind'] = test[c].astype(str)+'_'
        count+=1
    else :
        train['new_ind'] += train[c].astype(str)+'_'
        test['new_ind'] += test[c].astype(str)+'_'

In [243]:
train['new_ind'].value_counts()#high caldinality

0_2_1_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_      102
0_1_2_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       97
0_1_2_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       88
0_1_3_1_1_1_0_0_0_0_0_0_0_0_6_1_0_0_       82
0_1_3_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       79
0_1_1_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       78
0_2_0_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       74
1_2_1_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       72
0_1_3_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       69
0_2_1_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       68
0_1_1_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       63
1_1_2_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       63
0_1_2_1_1_1_0_0_0_0_0_0_0_0_6_1_0_0_       61
1_1_2_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       57
0_1_3_1_1_1_0_0_0_0_0_0_0_0_11_1_0_0_      56
0_1_2_1_1_1_0_0_0_0_0_0_0_0_9_1_0_0_       54
0_1_4_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_       53
0_1_2_1_1_1_0_0_0_0_0_0_0_0_3_0_0_1_       52
1_2_1_1_1_1_0_0_0_0_0_0_0_0_7_1_0_0_       51
0_1_3_1_1_1_0_0_0_0_0_0_0_0_3_0_0_1_       49
1_1_3_1_1_1_0_0_0_0_0_0_0_0_6_1_0_0_       48
1_1_1_1_1_1_0_0_0_0_0_0_0_0_8_1_0_

In [244]:
cat_count_features = []
for c in cat_features+['new_ind']:
    break
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

In [245]:
d = pd.concat([train[c],test[c]]).value_counts().to_dict()

In [246]:
d #high caldinality를 대체, 일종의 frequency encoding

{1: 72719, 2: 20717, 3: 4645, 4: 1892, 0: 27}

In [247]:
train[c]

0        2
1        1
2        4
3        1
4        2
5        1
6        1
7        1
8        1
9        1
10       2
11       1
12       1
13       1
14       1
15       1
16       2
17       1
18       1
19       1
20       1
21       1
22       3
23       1
24       1
25       1
26       1
27       1
28       2
29       2
        ..
49970    1
49971    2
49972    1
49973    2
49974    3
49975    3
49976    2
49977    3
49978    2
49979    1
49980    1
49981    2
49982    2
49983    3
49984    4
49985    1
49986    3
49987    1
49988    2
49989    1
49990    1
49991    1
49992    2
49993    2
49994    1
49995    2
49996    1
49997    1
49998    1
49999    1
Name: ps_ind_02_cat, Length: 50000, dtype: int64

In [248]:
train[c].apply(lambda x : d.get(x,0))

0        20717
1        72719
2         1892
3        72719
4        20717
5        72719
6        72719
7        72719
8        72719
9        72719
10       20717
11       72719
12       72719
13       72719
14       72719
15       72719
16       20717
17       72719
18       72719
19       72719
20       72719
21       72719
22        4645
23       72719
24       72719
25       72719
26       72719
27       72719
28       20717
29       20717
         ...  
49970    72719
49971    20717
49972    72719
49973    20717
49974     4645
49975     4645
49976    20717
49977     4645
49978    20717
49979    72719
49980    72719
49981    20717
49982    20717
49983     4645
49984     1892
49985    72719
49986     4645
49987    72719
49988    20717
49989    72719
49990    72719
49991    72719
49992    20717
49993    20717
49994    72719
49995    20717
49996    72719
49997    72719
49998    72719
49999    72719
Name: ps_ind_02_cat, Length: 50000, dtype: int64

- ind feature들을 하나로 묶어서, 새로운 카테고리를 만들어 낸것  

In [249]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,missing,new_ind
0,7,0,2,2,5,2,1,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,11,1,0,0,2,4,2,0,1,1,11,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1,1.0,2_2_5_2_1_0_1_0_0_0_0_0_0_0_11_0_1_0_
1,9,0,1,1,7,1,1,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,12,1,0,0,0,11,2,1,3,1,18,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0,2.0,1_1_7_1_1_0_0_1_0_0_0_0_0_0_3_0_0_1_
2,13,0,5,4,9,2,1,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,8,1,0,0,0,14,2,1,3,1,59,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0,3.0,5_4_9_2_1_0_0_1_0_0_0_0_0_0_12_1_0_0_
3,16,0,0,1,2,1,1,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,8,1,1,0,2,11,2,1,4,1,103,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0,0.0,0_1_2_1_1_1_0_0_0_0_0_0_0_0_8_1_0_0_
4,17,0,0,2,0,2,1,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,12,1,0,0,0,14,2,1,3,1,81,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0,2.0,0_2_0_2_1_1_0_0_0_0_0_0_0_0_9_1_0_0_


In [250]:
train_list = [train[num_features + cat_count_features].values, X_cat,]

In [251]:
test_list = [test[num_features + cat_count_features].values, X_t_cat]

In [252]:
train_list #sparse matrix로 저장되어 있음

[array([[0.6, 0.5, 0.2, ..., 0. , 1. , 1. ],
        [0.3, 0.1, 0.3, ..., 1. , 0. , 2. ],
        [0.5, 0.7, 0.1, ..., 1. , 0. , 3. ],
        ...,
        [0.1, 0. , 0.8, ..., 0. , 0. , 3. ],
        [0.2, 0.5, 0.8, ..., 1. , 0. , 2. ],
        [0.8, 0.3, 0.8, ..., 1. , 0. , 3. ]]),
 <50000x183 sparse matrix of type '<class 'numpy.float64'>'
 	with 700000 stored elements in Compressed Sparse Row format>]

In [253]:
X=ssp.hstack(train_list).tocsr()

In [254]:
ssp.hstack(train_list)

<50000x204 sparse matrix of type '<class 'numpy.float64'>'
	with 1503317 stored elements in COOrdinate format>

In [255]:
ssp.hstack(train_list).tocsr() #compressed sparse row format으로 바뀜

<50000x204 sparse matrix of type '<class 'numpy.float64'>'
	with 1503317 stored elements in Compressed Sparse Row format>

In [256]:
X_test = ssp.hstack(test_list).tocsr()

## Model Development

In [257]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {'objective' : 'binary',
          'boosting_type' : 'gbdt',
          'learning_rate' : learning_rate,
          'num_leaves' : num_leaves,
          'max_bin' : 256,
          'feature_fraction': feature_fraction,
          'verbosity':0,
          'drop_rate':0.1,
          'is_usbalance' : False,
          'max_drop' : 50,
          'min_child_samples': 10,
          'min_child_weight' : 150,
          'min_split_gain' : 0,
          'subsample' : 0.9
         }

In [258]:
# out of fold stacking
x_score = []
final_cv_train = np.zeros(len(train_label))
final_cv_pred = np.zeros(len(test_id))    

In [259]:
for s in np.arange(16):
    break

In [260]:
cv_train = np.zeros(len(train_label))
cv_pred = np.zeros(len(test_id))

In [261]:
params['seed'] = s #앙상블은 다양성이 있어서 좋은 것
# random num을 바꾸면서 다양성을 확보하려는 코드

In [262]:
kf = kfold.split(X, train_label)

In [263]:
best_trees = []
fold_scores = []

In [264]:
for i, (train_fold, validate) in enumerate(kf):
    break

In [265]:
validate

array([    2,     7,    23, ..., 49993, 49996, 49998])

In [266]:
X_train, label_train = X[train_fold, :], train_label[train_fold]

In [267]:
X_validate, label_validate = X[validate,:], train_label[validate]

In [268]:
dtrain = lgbm.Dataset(X_train, label_train)
dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)

In [269]:
bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
                            early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.157512	valid_0's gini: 0.118119
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.156657	valid_0's gini: 0.148781


In [270]:
bst.best_iteration

3

In [271]:
best_trees = []
fold_scores = []

In [272]:
best_trees.append(bst.best_iteration)

In [273]:
cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)

In [274]:
cv_train[validate] += bst.predict(X_validate)

In [275]:
score = Gini(label_validate, cv_train[validate])

In [276]:
score

0.14878108244019966

In [277]:
fold_scores.append(score)

In [278]:
kf

<generator object _BaseKFold.split at 0x7f8a91d21410>

In [279]:
for i, (train_fold, validate) in enumerate(kf):
    print('#'*30, '{} of {}'.format(i+1, 5))
    X_train, X_validate, label_train, label_validate = \
        X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
    dtrain = lgbm.Dataset(X_train, label_train)
    dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
    bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
                    early_stopping_rounds=100)
    best_trees.append(bst.best_iteration)
    cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
    cv_train[validate] += bst.predict(X_validate)

    score = Gini(label_validate, cv_train[validate])
    print(score)
    fold_scores.append(score)

cv_pred /= NFOLDS

############################## 1 of 5
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154076	valid_0's gini: 0.228955
Early stopping, best iteration is:
[26]	valid_0's binary_logloss: 0.154201	valid_0's gini: 0.242028
0.24202769596100102
############################## 2 of 5
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.155429	valid_0's gini: 0.182289
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.155086	valid_0's gini: 0.193815
0.1938151975879916
############################## 3 of 5
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154995	valid_0's gini: 0.19683
Early stopping, best iteration is:
[13]	valid_0's binary_logloss: 0.155003	valid_0's gini: 0.216065
0.21606454134495545
############################## 4 of 5
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154693

In [280]:
fold_scores

[0.14878108244019966,
 0.24202769596100102,
 0.1938151975879916,
 0.21606454134495545,
 0.21499864706873395]

In [281]:
cv_pred

array([0.02587586, 0.03045339, 0.02920774, ..., 0.05251369, 0.02505443,
       0.03028296])

In [282]:
final_cv_train += cv_train
final_cv_pred += cv_pred

In [283]:
print('cv score :')
print(Gini(train_label, cv_train))
print('current score', Gini(train_label, final_cv_train/(s+1.)), s+1)
print(fold_scores)
print(best_trees, np.mean(best_trees))

cv score :
0.19809839178931374
current score 0.19809839178931374 1
[0.14878108244019966, 0.24202769596100102, 0.1938151975879916, 0.21606454134495545, 0.21499864706873395]
[3, 26, 37, 13, 21] 20.0


In [284]:
x_score.append(Gini(train_label, cv_train))

In [285]:
print(x_score)

[0.19809839178931374]


#### cross validation stacking을 총 16번 진행한 것
#### 학습은 16x5 = 80번 진행하게 됨

In [289]:
for s in range(16):
    print('#'*30, 'random number outer iteration: {}'.format(s))
    cv_train = np.zeros(len(train_label))
    cv_pred = np.zeros(len(test_id))

    params['seed'] = s

    if cv_only:
        kf = kfold.split(X, train_label)

        best_trees = []
        fold_scores = []

        for i, (train_fold, validate) in enumerate(kf):
            print('#'*10, 'inner cross validation system {}'.format(i))
            X_train, X_validate, label_train, label_validate = \
                X[train_fold, :], X[validate, :], train_label[train_fold], train_label[validate]
            dtrain = lgbm.Dataset(X_train, label_train)
            dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
            bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
                            early_stopping_rounds=100)
            best_trees.append(bst.best_iteration)
            cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
            cv_train[validate] += bst.predict(X_validate)

            score = Gini(label_validate, cv_train[validate])
            print(score)
            fold_scores.append(score)

        cv_pred /= NFOLDS
        final_cv_train += cv_train
        final_cv_pred += cv_pred

        print("cv score:")
        print(Gini(train_label, cv_train))
        print("current score:", Gini(train_label, final_cv_train / (s + 1.)), s+1)
        print(fold_scores)
        print(best_trees, np.mean(best_trees))

        x_score.append(Gini(train_label, cv_train))

print(x_score)

############################## random number outer iteration: 0
########## inner cross validation system 0
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.157512	valid_0's gini: 0.118119
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.156657	valid_0's gini: 0.148781
0.14878108244019966
########## inner cross validation system 1
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154076	valid_0's gini: 0.228955
Early stopping, best iteration is:
[26]	valid_0's binary_logloss: 0.154201	valid_0's gini: 0.242028
0.24202769596100102
########## inner cross validation system 2
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.155429	valid_0's gini: 0.182289
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.155086	valid_0's gini: 0.193815
0.1938151975879916
########## inner cross validation system 3
Training until vali

[100]	valid_0's binary_logloss: 0.154657	valid_0's gini: 0.197467
Early stopping, best iteration is:
[15]	valid_0's binary_logloss: 0.154775	valid_0's gini: 0.216395
0.21639469060986274
cv score:
0.19170902390189545
current score: 0.19956582680024038 5
[0.15432627066152524, 0.24112320183003094, 0.19588177673760526, 0.21806415673807983, 0.21639469060986274]
[1, 55, 25, 6, 15] 20.4
############################## random number outer iteration: 5
########## inner cross validation system 0
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.157377	valid_0's gini: 0.122103
Early stopping, best iteration is:
[19]	valid_0's binary_logloss: 0.156031	valid_0's gini: 0.146298
0.14629782463222052
########## inner cross validation system 1
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154104	valid_0's gini: 0.227021
Early stopping, best iteration is:
[29]	valid_0's binary_logloss: 0.154136	valid_0's gini

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.155137	valid_0's gini: 0.192642
Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.15543	valid_0's gini: 0.215906
0.2159062739138401
########## inner cross validation system 4
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154406	valid_0's gini: 0.205339
Early stopping, best iteration is:
[27]	valid_0's binary_logloss: 0.154281	valid_0's gini: 0.224541
0.22454120881146827
cv score:
0.19650181434249192
current score: 0.20205523356970292 10
[0.1399765476381051, 0.23844026520514602, 0.19717048938551662, 0.2159062739138401, 0.22454120881146827]
[26, 25, 24, 8, 27] 22.0
############################## random number outer iteration: 10
########## inner cross validation system 0
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.157597	valid_0's gini: 0.112872
Early stopping, best iterati

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.155496	valid_0's gini: 0.181522
Early stopping, best iteration is:
[16]	valid_0's binary_logloss: 0.155388	valid_0's gini: 0.189134
0.18913442127156566
########## inner cross validation system 3
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.155254	valid_0's gini: 0.188693
Early stopping, best iteration is:
[21]	valid_0's binary_logloss: 0.154676	valid_0's gini: 0.217384
0.2173840038710284
########## inner cross validation system 4
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.154921	valid_0's gini: 0.189328
Early stopping, best iteration is:
[10]	valid_0's binary_logloss: 0.155131	valid_0's gini: 0.211458
0.21145776783926373
cv score:
0.19861070284530505
current score: 0.20208989203785233 15
[0.1515471714335978, 0.2401858992815523, 0.18913442127156566, 0.2173840038710284, 0.2114577678392637

## 결국 다양성을 위한 작업

## 정리
- 카테고리 인코딩 후 왜 합치는가?(1_3_0_)이런식으로?
    - ind라는 feature는 카테고리 피쳐였음, 합쳐야 한다는 의견이 있었음
    - 합쳐보니까 성능이 올라갔음
    - ps_ind_01이런게 뭘 의미하는지 몰라서 그런 것
    - 타이타닉으로 생각해보면 (남자_1_C, 남자_1_S, 여자_2_C, 남자_2_S...)이런게 만들어지는것임
- 포르투가 어려웠던 이유 ==> 비식별화 때문